In [1]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Model, load_model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D, AveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from imagenet_utils import preprocess_input

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
img_width, img_height = 200, 200
train_data_dir = "images/train"
validation_data_dir = "images/dev"
nb_train_samples = 32488
nb_validation_samples = 5723
batch_size = 32
epochs = 50

In [3]:
# load the original ResNet without top layer
model = applications.ResNet50(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))

In [4]:
# adding custom Layers 
x = model.output
x = Flatten(name='flatten')(x)
out = Dense(18, activation="softmax", name='output_layer')(x)

# create the final model
model_final = Model(inputs=model.input, outputs=out)

In [5]:
model_final.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 200, 200, 3)  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 100, 100, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 100, 100, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 100, 100, 64) 0           bn_conv1[0][0]                   
__________________________________________________________________________________________________
max_poolin

In [14]:
# freeze layers to stage 3d
for layer in model_final.layers[:79]:
    layer.trainable = False

In [6]:
def print_layer_trainable(model_name):
    for layer in model_name.layers:
        print("{0}:\t{1}".format(layer.trainable, layer.name))

In [15]:
print_layer_trainable(model_final)

False:	input_1
False:	conv1
False:	bn_conv1
False:	activation_1
False:	max_pooling2d_1
False:	res2a_branch2a
False:	bn2a_branch2a
False:	activation_2
False:	res2a_branch2b
False:	bn2a_branch2b
False:	activation_3
False:	res2a_branch2c
False:	res2a_branch1
False:	bn2a_branch2c
False:	bn2a_branch1
False:	add_1
False:	activation_4
False:	res2b_branch2a
False:	bn2b_branch2a
False:	activation_5
False:	res2b_branch2b
False:	bn2b_branch2b
False:	activation_6
False:	res2b_branch2c
False:	bn2b_branch2c
False:	add_2
False:	activation_7
False:	res2c_branch2a
False:	bn2c_branch2a
False:	activation_8
False:	res2c_branch2b
False:	bn2c_branch2b
False:	activation_9
False:	res2c_branch2c
False:	bn2c_branch2c
False:	add_3
False:	activation_10
False:	res3a_branch2a
False:	bn3a_branch2a
False:	activation_11
False:	res3a_branch2b
False:	bn3a_branch2b
False:	activation_12
False:	res3a_branch2c
False:	res3a_branch1
False:	bn3a_branch2c
False:	bn3a_branch1
False:	add_4
False:	activation_13
False:	res3b_branch

In [16]:
# compile the model
model_final.compile(loss="categorical_crossentropy", optimizer=optimizers.SGD(lr=1e-4, momentum=0.9), metrics=["accuracy"])

In [ ]:
# load the saved model
# model_final = load_model('models/ResNet50_trained_f3d.h5')
# model_final.summary()

In [17]:
# initiate the train and validation generators with data augumentation
train_datagen = ImageDataGenerator(
        rotation_range=30,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        preprocessing_function=preprocess_input,
        fill_mode='nearest')

valid_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical')

validation_generator = valid_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical')

# save the model according to the conditions
checkpoint = ModelCheckpoint("models/ResNet50_trained_f3d.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')

Found 32488 images belonging to 18 classes.
Found 5723 images belonging to 18 classes.


In [18]:
print(train_generator.class_indices)
print(validation_generator.class_indices)

{'BabyBibs': 0, 'BabyHat': 1, 'BabyPants': 2, 'BabyShirt': 3, 'PackageFart': 4, 'womanshirtsleeve': 5, 'womencasualshoes': 6, 'womenchiffontop': 7, 'womendollshoes': 8, 'womenknittedtop': 9, 'womenlazyshoes': 10, 'womenlongsleevetop': 11, 'womenpeashoes': 12, 'womenplussizedtop': 13, 'womenpointedflatshoes': 14, 'womensleevelesstop': 15, 'womenstripedtop': 16, 'wrapsnslings': 17}
{'BabyBibs': 0, 'BabyHat': 1, 'BabyPants': 2, 'BabyShirt': 3, 'PackageFart': 4, 'womanshirtsleeve': 5, 'womencasualshoes': 6, 'womenchiffontop': 7, 'womendollshoes': 8, 'womenknittedtop': 9, 'womenlazyshoes': 10, 'womenlongsleevetop': 11, 'womenpeashoes': 12, 'womenplussizedtop': 13, 'womenpointedflatshoes': 14, 'womensleevelesstop': 15, 'womenstripedtop': 16, 'wrapsnslings': 17}


In [19]:
model_final.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size,
    callbacks = [checkpoint])

Epoch 1/50
1015/1015 [==============================] - 9520s 9s/step - loss: 1.9575 - acc: 0.3932 - val_loss: 1.2188 - val_acc: 0.5827

Epoch 00001: val_acc improved from -inf to 0.58269, saving model to models/ResNet50_trained_f3d.h5
Epoch 2/50
1015/1015 [==============================] - 9445s 9s/step - loss: 1.2091 - acc: 0.6011 - val_loss: 0.9394 - val_acc: 0.6730

Epoch 00002: val_acc improved from 0.58269 to 0.67299, saving model to models/ResNet50_trained_f3d.h5
Epoch 3/50
1015/1015 [==============================] - 9464s 9s/step - loss: 1.0026 - acc: 0.6619 - val_loss: 0.8401 - val_acc: 0.7076

Epoch 00003: val_acc improved from 0.67299 to 0.70761, saving model to models/ResNet50_trained_f3d.h5
Epoch 4/50
1015/1015 [==============================] - 9425s 9s/step - loss: 0.8878 - acc: 0.6976 - val_loss: 0.7508 - val_acc: 0.7449

Epoch 00004: val_acc improved from 0.70761 to 0.74486, saving model to models/ResNet50_trained_f3d.h5
Epoch 5/50
1015/1015 [=========================

1015/1015 [==============================] - 6096s 6s/step - loss: 0.2575 - acc: 0.9107 - val_loss: 0.6211 - val_acc: 0.8011

Epoch 00042: val_acc did not improve
Epoch 43/50
1015/1015 [==============================] - 6096s 6s/step - loss: 0.2526 - acc: 0.9127 - val_loss: 0.6185 - val_acc: 0.8035

Epoch 00043: val_acc did not improve
Epoch 44/50
1015/1015 [==============================] - 6097s 6s/step - loss: 0.2458 - acc: 0.9163 - val_loss: 0.6238 - val_acc: 0.8076

Epoch 00044: val_acc did not improve
Epoch 45/50
1015/1015 [==============================] - 6100s 6s/step - loss: 0.2363 - acc: 0.9196 - val_loss: 0.6239 - val_acc: 0.8076

Epoch 00045: val_acc did not improve
Epoch 46/50
1015/1015 [==============================] - 6093s 6s/step - loss: 0.2334 - acc: 0.9198 - val_loss: 0.6326 - val_acc: 0.8043

Epoch 00046: val_acc did not improve
Epoch 47/50
1015/1015 [==============================] - 6096s 6s/step - loss: 0.2240 - acc: 0.9219 - val_loss: 0.6491 - val_acc: 0.8035


In [ ]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils.vis_utils import plot_model

# plot_model(model, to_file='model.png')
SVG(model_to_dot(model_final).create(prog='dot', format='svg'))